In [ ]:
import os
from qiskit import IBMQ
from qiskit.providers.ibmq.runtime import UserMessenger
from qiskit.providers.ibmq.runtime.exceptions import QiskitRuntimeError, RuntimeProgramNotFound
from program_backend import main

this_dir = os.getcwd()

program_path = os.path.join(this_dir, 'program_backend.py')

In [ ]:
provider = IBMQ.load_account()
backend = provider.get_backend('ibmq_lima')

In [ ]:
cred = provider.credentials
inputs = {'api_token': cred.access_token, 'hub': cred.hub, 'group': cred.group, 'project': cred.project, 'qubits': [3, 4]}

In [ ]:
ibmq_backend = main(backend, UserMessenger(), **inputs)

In [ ]:
program_id = ''

if True:
    json_schema = 'https://json-schema.org/draft/2019-09/schema'
    
    metadata = {
        'name': 'backend-dumper-nativeq',
        'max_execution_time': 60,
        'description': 'Dump the parameters of the mysterious RuntimeBackend object.',
        'is_public': False,
        'spec': {
            'backend_requirements': {'min_num_qubits': 2},
            'parameters': {
                '$schema': json_schema,
                'properties': {
                    'api_token': {
                        'type': 'string',
                        'description': 'IBMQ API token.'
                    },
                    'hub': {
                        'type': 'string',
                        'description': 'Hub name.'
                    },
                    'group': {
                        'type': 'string',
                        'description': 'Group name.'
                    },
                    'project': {
                        'type': 'string',
                        'description': 'Project name.'
                    },
                    'qubits': {
                        'type': 'array',
                        'description': 'Physical qubits.'
                    }
                },
                'required': ['api_token', 'hub', 'group', 'project', 'qubits']
            },
            'return_values': {
                '$schema': json_schema,
                'type': 'object',
                'description': 'Dump of the backend parameters.'
            }
            #'interim_results': {'$schema': json_schema, 'type': str, 'description': 'Interim message'}
        }
    }

    ## Upload or update the program
    
    if not program_id:
        program_id = provider.runtime.upload_program(
            data=program_path,
            metadata=metadata
        )

        print('Uploaded program with id', program_id)
        
    else:
        provider.runtime.update_program(program_id, data=program_path, metadata=metadata)

In [ ]:
options = {'backend_name': backend.name()}
job = provider.runtime.run(
    program_id=program_id,
    options=options,
    inputs=inputs
)

print('job id:', job.job_id())
runtime_backend = job.result()